In [ ]:
import xarray as xr
from emit_tools import emit_xarray
import os
from fsspec.implementations.http import HTTPFileSystem
import numpy as np
import math

In [ ]:
# Generate a user token and set it as an environment variable
# Tokens can be generated at https://urs.earthdata.nasa.gov/users/<username>/user_tokens
token = os.environ.get('EARTHDATA_TOKEN')
if token is None:
    raise ValueError('Please set the EARTHDATA_TOKEN environment variable')

In [ ]:
%%time
# Loading data can take around 4 minutes on a 100Mbps connection

# kernel = "EMIT_L2A_RFL_001_20230123T004529_2302216_003" # Hobart
kernel = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + kernel + "/" + kernel + ".nc"
s3_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(s3_url))
ds

## Plot bands as an RGB image

In [ ]:
# 650 = Red, 560 = Green and 470 = Blue
# Change these to other bands and this will select the closest band
band_1 = 650
band_2 = 560
band_3 = 470

ds["band_1"] = ds.reflectance.sel(bands=band_1, method="nearest")
ds["band_2"] = ds.reflectance.sel(bands=band_2, method="nearest")
ds["band_3"] = ds.reflectance.sel(bands=band_3, method="nearest")

In [ ]:
ds.drop_dims("bands").to_array().plot.imshow(x="longitude", y="latitude", robust=True, vmin=0, vmax=0.2)

## View a single band on an interactive map

In [ ]:
import folium
import odc.geo.xr

m = folium.Map(control_scale=True, tiles=None)

band = 450
ds.reflectance.sel(bands=450, method="nearest").odc.add_to(m, opacity=0.7, title=f"band_{band}")

# Zoom map
m.fit_bounds(ds.odc.map_bounds())

tile = folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    overlay = True,
    control = True
).add_to(m)

folium.LayerControl().add_to(m)
display(m)

## Plot the spectra at a point

In [ ]:
# Using matplotlib to plot the reflectance of a single location
example = ds['reflectance'].sel(longitude=149.2,latitude=-34.9, method='nearest')
example.plot(x='bands', size=2, aspect=4)

## Exporting data

In [ ]:
ds.red.rio.to_raster("red.tif")
ds.green.rio.to_raster("green.tif")
ds.blue.rio.to_raster("blue.tif")

In [ ]:
!gdal_merge.py -separate red.tif green.tif blue.tif -o rgb.tif 